In [59]:
from ase import Atoms
from ase.io import read, write
import xcquinox as xce
import torch, jax, optax
import numpy as np
import equinox as eqx
import jax.numpy as jnp
import pyscfad as psa
import os, sys, pickle
from pyscf import dft, scf, gto, df
from pyscf.pbc import scf as scfp
from pyscf.pbc import gto as gtop
from pyscf.pbc import dft as dftp
# from pyscfad.pbc import scf as scfp
# from pyscfad.pbc import gto as gtop
# from pyscfad.pbc import dft as dftp
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'
import pandas as pd
from mp_api.client import MPRester
from mldftdat.density import get_exchange_descriptors2
from mldftdat.analyzers import RKSAnalyzer
from emmet.core.summary import HasProps
from ase.build import bulk
from pymatgen.io.ase import AseAtomsAdaptor

In [61]:
gtop.M?

Signature: gtop.M(**kwargs)
Docstring:
This is a shortcut to build up Cell object.

Examples:

>>> from pyscf.pbc import gto
>>> cell = gto.M(a=numpy.eye(3)*4, atom='He 1 1 1', basis='6-31g')
File:      ~/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/pbc/gto/cell.py
Type:      function

In [3]:
borlido = pd.read_excel('/home/awills/Documents/Research/datasets/borlido2019/gg_direct_gaps.xlsx')

In [4]:
borlido

,Unnamed: 0,Material,MP_ID,ICSD_ID,LDA,LDA + SOC,PBE,PBE + SOC,PBE_SOL,HLE16,...,BJ,MBJ,SCAN,HSE06,HSE14,HSE06_mix,PBE0,PBE0_mix,Experimental,"DOI for the experimental references (missing entries refer to values as provided by 10.1007/978-3-642-18865-7, Semiconductors: Data Handbook, Otfried Madelung, Springer-Verlag Berlin Heidelberg, 2004)"
0,3,Ag2MoO4,mp-19318,238013,2.0985,2.0183,2.2497,2.1712,2.1260,4.1954,...,2.9429,3.9113,2.6264,3.9265,4.6616,5.3773,4.6367,6.0903,3.4000,10.1039/C5TA05248C
1,12,AgAlS2,mp-5782,604692,1.9100,1.8964,2.1233,2.1073,2.0006,3.6409,...,2.6005,3.1894,2.5863,3.3278,3.6784,3.6794,3.8922,3.7563,3.1300,10.1088/0022-3727/4/8/319
2,13,AgAlSe2,mp-14091,604704,0.8204,0.7886,1.0503,1.0190,0.9402,2.1914,...,1.4828,2.0714,1.4602,2.0664,2.3968,2.2946,2.4568,2.3324,2.5500,10.1088/0022-3727/4/8/319
3,14,AgAlTe2,mp-14092,28746,0.9898,0.8251,1.1699,1.0092,1.0847,1.8419,...,1.4958,1.9638,1.4711,2.0440,2.2627,2.1838,2.6443,2.2097,2.3500,10.1103/PhysRevB.9.5203
4,20,AgGaS2,mp-5342,605187,1.0165,1.0083,1.2432,1.2304,1.1288,2.5881,...,1.7559,2.5022,1.6440,2.4202,2.8348,2.8872,2.9515,3.0952,2.7140,10.1103/PhysRevB.4.4455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,464,ZnSe,mp-1190,77091,1.0489,0.9191,1.3076,1.1817,1.1876,2.6323,...,1.8166,2.6973,1.7126,2.4218,2.8588,2.7623,3.0423,2.9850,2.8222,NaN
130,465,ZnSiAs2,mp-3595,68323,1.0431,0.9825,1.1391,1.0791,1.0387,1.4475,...,1.4644,1.7512,1.3640,1.7498,1.7668,1.8272,2.3720,1.9801,1.9300,NaN
131,466,ZnSiP2,mp-4763,23680,1.1770,1.1726,1.3004,1.2949,1.1787,2.3236,...,1.5554,1.9131,1.5423,1.9822,2.0170,2.0758,2.5910,2.1567,2.0000,NaN
132,468,ZnSnO3,mp-13334,245943,1.5268,1.5169,1.6087,1.5944,1.5224,2.5159,...,2.3269,3.2248,2.1172,3.3615,4.3674,4.9717,3.8854,5.6142,3.9000,10.1039/B921010E


In [6]:
with open('/home/awills/Documents/Research/datasets/borlido2019/ggdirect.pkl', 'rb') as f:
    bstruc_list = pickle.load(f)

In [8]:
smalls = [i for i in bstruc_list if len(i[-1].structure.as_dict()['sites']) <= 4]

In [30]:
smalls[0]

(22,
 'mp-22894',
 <pymatgen.electronic_structure.bandstructure.BandStructureSymmLine at 0x75cef14962c0>)

In [11]:
smallids = [i[1] for i in smalls]

In [12]:
smalldf = borlido[borlido['MP_ID'].isin(smallids)]

In [37]:
traj = []
for small in smalls:
    datidx, mpid, bstruc = small
    at = AseAtomsAdaptor.get_atoms(bstruc.structure)
    at.info['borlido_idx'] = datidx
    at.info['mp_id'] = mpid
    at.info['reference'] = smalldf[smalldf['MP_ID']==mpid]['Experimental'].values[0]
    traj.append(at)

In [38]:
traj[2].info

{'borlido_idx': 31, 'mp_id': 'mp-661', 'reference': 6.11}

In [45]:
tt = traj[2]

In [47]:
from pyscf.pbc.tools import pyscf_ase

In [52]:
pyscf_ase.ase_atoms_to_pyscf(tt)

[['Al', array([-1.564023  ,  0.90322835, -2.50702574])],
 ['Al', array([-1.56402300e+00, -9.03228350e-01, -6.86736886e-04])],
 ['N', array([-1.564023  ,  0.90323919, -0.58930546])],
 ['N', array([-1.564023  , -0.90323919, -3.09564446])]]

In [41]:
write('/home/awills/Documents/Research/datasets/borlido2019/smalls.traj', traj)

In [17]:
test = smalls[0][-1]

In [58]:
gtop

<module 'pyscfad.pbc.gto' from '/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscfad/pbc/gto/__init__.py'>

In [22]:
test.as_dict()['lattice_rec']

{'@module': 'pymatgen.core.lattice',
 '@class': 'Lattice',
 'matrix': [[-1.3444316765507018, -0.7762705798519046, -0.0],
  [-1.3444316765507018, 0.7762705798519046, -0.0],
  [-0.0, 0.0, -0.8134402974930758]],
 'pbc': (True, True, True)}

In [26]:
at  = AseAtomsAdaptor.get_atoms(test.structure)

In [66]:
mol.pbc

NameError: name 'mol' is not defined

In [69]:
gtop.Cell(a=np.array(at.cell), charge=0).spin

0

In [70]:
from pyscf.pbc import cc

In [ ]:
cc.

In [3]:
image = np.array([
[[1, 2, 3], [4, 5, 6], [-1, 8, 9]],
[[10, 11, 12], [13, 14, 15], [16, 17, 18]],
[[19, 20, 21], [22, 23, 24], [25, 26, 27]]
],
dtype=np.float32)
mask = np.array(
[[1, 4, 7],
[10, 4, 16],
[19, 22, 255]],
dtype=np.uint8)

In [62]:
with MPRester(api_key = '') as mpr:
    docs = mpr.materials.summary.search(
        band_gap=(0, 5), is_stable=True, fields=["material_id"], is_gap_direct = True, num_elements=(0, 4), num_sites=(0, 4),
        has_props=[HasProps.bandstructure]
    )
    gamma_directs = [mpr.get_bandstructure_by_material_id(str(i.material_id)) for i in docs]

Retrieving SummaryDoc documents:   0%|          | 0/102 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [70]:
t = gamma_directs[10]

In [71]:
t.get_band_gap()

{'direct': True, 'energy': 4.4292, 'transition': '\\Gamma-\\Gamma'}

In [73]:
gamgam = [i for i in gamma_directs if i.get_band_gap()['transition'] == '\\Gamma-\\Gamma']

In [160]:
len(borlido_gg_direct[0][-1].structure.as_dict()['sites'])

14

In [173]:
smalls = [(i[1], i[-1]) for i in borlido_gg_direct if len(i[-1].structure.as_dict()['sites']) < 4]

In [189]:
[i[1].structure.as_dict()['sites'][0]['species'] for i in smalls]

[[{'element': 'Ag', 'occu': 1}],
 [{'element': 'Ar', 'occu': 1}],
 [{'element': 'Cd', 'occu': 1}],
 [{'element': 'Cd', 'occu': 1}],
 [{'element': 'Cu', 'occu': 1}],
 [{'element': 'Cu', 'occu': 1}],
 [{'element': 'Cu', 'occu': 1}],
 [{'element': 'Ga', 'occu': 1}],
 [{'element': 'In', 'occu': 1}],
 [{'element': 'K', 'occu': 1}],
 [{'element': 'K', 'occu': 1}],
 [{'element': 'Kr', 'occu': 1}],
 [{'element': 'Li', 'occu': 1}],
 [{'element': 'Li', 'occu': 1}],
 [{'element': 'Mg', 'occu': 1}],
 [{'element': 'Na', 'occu': 1}],
 [{'element': 'Na', 'occu': 1}],
 [{'element': 'Na', 'occu': 1}],
 [{'element': 'Na', 'occu': 1}],
 [{'element': 'Rb', 'occu': 1}],
 [{'element': 'Rb', 'occu': 1}],
 [{'element': 'Rb', 'occu': 1}],
 [{'element': 'Xe', 'occu': 1}],
 [{'element': 'Y', 'occu': 1}],
 [{'element': 'Zn', 'occu': 1}],
 [{'element': 'Zn', 'occu': 1}]]

In [4]:
mpr = MPRester(api_key = '')

In [164]:
# mp = mpr.get_bandstructure_by_material_id('mp-1266')
mp = smalls[0]
mps = mp.structure.as_dict()
mp.get_band_gap()

{'direct': True, 'energy': 1.3674, 'transition': '\\Gamma-\\Gamma'}

In [165]:
rets = mps
at_coor_xyz = [ (i['species'][0]['element'], i['xyz']) for i in rets['sites']]
at_coor_abc = [ (i['species'][0]['element'], [rets['lattice']['a']*j for j in i['abc']]) for i in rets['sites']]
lat = np.array(rets['lattice']['matrix'])
if np.linalg.det(lat) < 1:
    print('left handed array, switching')
    lat = -np.array([lat[-1], lat[1], lat[0]])

In [166]:
np.linalg.det(lat)

73.21955895634993

In [72]:
traj

[MSONAtoms(symbols='Ag2I2', pbc=True, cell=[[-2.336744, -4.047033, 0.0], [-2.336744, 4.047033, 0.0], [0.0, 0.0, -7.724212]]),
 MSONAtoms(symbols='AgI', pbc=True, cell=[[0.0, 3.320466, 3.320466], [3.320466, 0.0, 3.320466], [3.320466, 3.320466, 0.0]]),
 MSONAtoms(symbols='Al2N2', pbc=True, cell=[[-1.564023, -2.709322, 0.0], [-1.564023, 2.709322, 0.0], [0.0, 0.0, -5.012678]]),
 MSONAtoms(symbols='Ar', pbc=True, cell=[[0.0, 2.820386, 2.820386], [2.820386, 0.0, 2.820386], [2.820386, 2.820386, 0.0]]),
 MSONAtoms(symbols='Be2O2', pbc=True, cell=[[-1.35488, -2.346663, 0.0], [-1.35488, 2.346663, 0.0], [0.0, 0.0, -4.402034]]),
 MSONAtoms(symbols='Cd2S2', pbc=True, cell=[[-2.104551, -3.645352, 0.0], [-2.104551, 3.645352, 0.0], [0.0, 0.0, -6.829614]]),
 MSONAtoms(symbols='CdSe', pbc=True, cell=[[-3.107208, -3.107208, 0.0], [-3.107208, 0.0, -3.107208], [0.0, -3.107208, -3.107208]]),
 MSONAtoms(symbols='Cd2Se2', pbc=True, cell=[[-2.19879, -3.807937, 0.0], [-2.19879, 3.807937, 0.0], [0.0, 0.0, -7.147

In [168]:
at_coor_abc

[('Ag', [0.0, 0.0, 0.0]),
 ('I', [3.5218860379490566, 3.5218860379490566, 3.5218860379490566])]

In [ ]:
cell = gtop.Cell(verbose=9)
cell.atom = at_coor_abc
cell.a = lat
cell.basis = 'def2-svp'
cell.pseudo = 'gth-pbe'
cell.exp_to_discard = 0.1
cell.build()
# kpts = cell.make_kpts([2,2,2])
mf = dftp.RKS(cell, xc='pbe')
# mf = scfp.RHF(cell)
e = mf.kernel()

arg.atm = [[11 20  1 23  0  0]
 [ 7 24  1 27  0  0]]
arg.bas = [[  0   0   3   1   0  28  31   0]
 [  0   0   1   1   0  34  35   0]
 [  0   0   1   1   0  36  37   0]
 [  0   0   1   1   0  38  39   0]
 [  0   0   1   1   0  40  41   0]
 [  0   1   4   1   0  42  46   0]
 [  0   1   1   1   0  50  51   0]
 [  0   1   1   1   0  52  53   0]
 [  0   2   4   1   0  54  58   0]
 [  0   2   1   1   0  62  63   0]
 [  0   3   1   1   0  64  65   0]
 [  1   0   6   1   0  66  72   0]
 [  1   0   2   1   0  78  80   0]
 [  1   0   1   1   0  82  83   0]
 [  1   0   1   1   0  84  85   0]
 [  1   1   4   1   0  86  90   0]
 [  1   1   1   1   0  94  95   0]
 [  1   1   1   1   0  96  97   0]
 [  1   1   1   1   0  98  99   0]
 [  1   2   5   1   0 100 105   0]
 [  1   2   1   1   0 110 111   0]]
arg.env = [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e

In [179]:
mf.mo_occ, mf.mo_energy

(array([2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0.]),
 Array([-0.09812588,  0.11092826,  0.11156689,  0.11156689,  0.11292458,
         0.11292458,  0.25918294,  0.25918294,  0.26567128,  0.54306331,
         0.56076239,  0.56076239,  0.69535016], dtype=float64))

In [180]:
mf.mo_energy - mf.mo_energy[mf.mo_occ == 0][0], mf.mo_energy - mf.mo_energy[mf.mol.nelectron//2-1], mf.mo_energy, mf.mo_occ

(Array([-0.64118919, -0.43213505, -0.43149642, -0.43149642, -0.43013873,
        -0.43013873, -0.28388037, -0.28388037, -0.27739204,  0.        ,
         0.01769908,  0.01769908,  0.15228685], dtype=float64),
 Array([-0.36379716, -0.15474302, -0.15410438, -0.15410438, -0.1527467 ,
        -0.1527467 , -0.00648833, -0.00648833,  0.        ,  0.27739204,
         0.29509111,  0.29509111,  0.42967889], dtype=float64),
 Array([-0.09812588,  0.11092826,  0.11156689,  0.11156689,  0.11292458,
         0.11292458,  0.25918294,  0.25918294,  0.26567128,  0.54306331,
         0.56076239,  0.56076239,  0.69535016], dtype=float64),
 array([2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0.]))

In [16]:
a1 = bulk('Si', a=3.867114, b=3.867114, c=3.867114, alpha=60)

In [17]:
a1.cell

Cell([[0.0, 1.933557, 1.933557], [1.933557, 0.0, 1.933557], [1.933557, 1.933557, 0.0]])

In [62]:
gtop.Cell?

Init signature: gtop.Cell(**kwargs)
Docstring:     
A Cell object holds the basic information of a crystal.

Attributes:
    a : (3,3) ndarray
        Lattice primitive vectors. Each row represents a lattice vector
        Reciprocal lattice vectors are given by  b1,b2,b3 = 2 pi inv(a).T
    mesh : (3,) list of ints
        The number G-vectors along each direction.
        The default value is estimated based on :attr:`precision`
    pseudo : dict or str
        To define pseudopotential.
    precision : float
        To control Ewald sums and lattice sums accuracy
    rcut : float
        Cutoff radius (unit Bohr) in lattice summation. The default value
        is estimated based on the required :attr:`precision`.
    ke_cutoff : float
        If set, defines a spherical cutoff of planewaves, with .5 * G**2 < ke_cutoff
        The default value is estimated based on :attr:`precision`
    dimension : int
        Periodic dimensions. Default is 3
    low_dim_ft_type : str
        For s

In [ ]:
gtop

In [153]:
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []

cell = gtop.Cell()
a = 5.43
cell.atom = [['Si', [0,0,0]],
              ['Si', [a/4,a/4,a/4]]]
cell.a = jnp.asarray([[0, a/2, a/2],
                     [a/2, 0, a/2],
                     [a/2, a/2, 0]])
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build()
kpts = cell.make_kpts([2,2,2])
mf = dftp.RKS(cell, xc='pbe0')
mf2 = dftp.KRKS(cell, xc='pbe0', kpts=kpts)
e = mf.kernel()
# e2 = mf2.kernel()

/home/awills/anaconda3/envs/pyscfad/lib/python3.10/site-packages/pyscf/pbc/gto/cell.py:243: UserWarning: Function cell.dumps drops attribute a because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -7.21836790602506


In [19]:
mf

RKS-KohnShamDFT object of <class 'pyscf.pbc.dft.rks.RKS'>

In [20]:
mfs = []
mols = []
energies = []
dms = []
ao_evals = []
gws = []
eris = []
mo_occs = []
hcs = []
vs = []
ts = []
ss = []
hologaps = []
ogds = []

mfs.append(mf)
dm = mf.make_rdm1()
dmj = jnp.array(dm)
dmj.flags = dm.flags
ao_eval = jnp.array(mf._numint.eval_ao(mf.mol, mf.grids.coords, deriv=2))
energies.append(jnp.array(mf.get_veff().exc))
dms.append(dmj)
ogds.append(dm.shape)
ao_evals.append(jnp.array(ao_eval))
gws.append(jnp.array(mf.grids.weights))
ts.append(jnp.array(mf.mol.intor('int1e_kin')))
vs.append(jnp.array(mf.mol.intor('int1e_nuc')))
mo_occs.append(jnp.array(mf.mo_occ))
hcs.append(jnp.array(mf.get_hcore()))
eris.append(jnp.array(mf.mol.intor('int2e')))
ss.append(jnp.linalg.inv(jnp.linalg.cholesky(mf.mol.intor('int1e_ovlp'))))
hologaps.append(jnp.array(mf.mo_energy[mf.mo_occ == 0][0] - mf.mo_energy[mf.mo_occ > 1][-1]))

In [21]:
class Band_gap_1shot_loss(eqx.Module):
    def __init__(self):
        super().__init__()

    def __call__(self, model, ao_eval, gw, dm, eri, mo_occ, hc, s, ogd, refgap, mf, alpha0=0.7):
        vgf = lambda x: model(x, ao_eval, gw, mf)
        dmp, moep, mocp = xce.utils.get_dm_moe(dm, eri, vgf, mo_occ, hc, s, ogd, alpha0)
        
        efermi = moep[mf.mol.nelectron//2-1]
        moep -= efermi
        # print(moep)
        moep_gap = jnp.min(moep)
        # print(moep_gap)
        loss = jnp.sqrt( (moep_gap - refgap)**2)
        # print(loss)
        return jnp.sqrt( (moep_gap - refgap)**2)


In [43]:
xce.net.eX?

Init signature:
xce.net.eX(
    n_input,
    n_hidden=16,
    depth=3,
    use=[],
    ueg_limit=False,
    lob=1.804,
    seed=92017,
)
Docstring:      eX(n_input, n_hidden=16, depth=3, use=[], ueg_limit=False, lob=1.804, seed=92017)
Init docstring:
__init__ Local exchange model based on MLP.

Receives density descriptors in this order : [rho, s, alpha, nl], where the input may be truncated depending on XC-level of approximation.

The MLP generated is hard-coded to have one output value -- the predicted exchange energy given a specific input from the grid.

:param n_input: Input dimensions (LDA: 1, GGA: 2, meta-GGA: 3, ...)
:type n_input: int
:param n_hidden: Number of hidden nodes (three hidden layers used by default), defaults to 16
:type n_hidden: int, optional
:param depth: Depth of the MLP, defaults to 3
:type depth: int, optional
:param use: Only these indices are used as input to the model (can be used to omit density as input to enforce uniform density scaling). These indices 

In [44]:
#update docs, only input =2 ??? for MGGA? holdover from sebastian for some reason
xnet = xce.net.eX(n_input = 2, n_hidden=32, depth=4, use = [1, 2], ueg_limit=True, lob=1.174)
# I guess use default LOB
cnet = xce.net.eC(n_input = 4, n_hidden=32, depth=4, use = [2, 3], ueg_limit=True)
blankxc = xce.xc.eXC(grid_models = [xnet, cnet], level=3)
p = '/home/awills/Documents/Research/xcquinox/models/pretrained/scan'
# xc = eqx.tree_deserialise_leaves(os.path.join(p, 'xc.eqx'), blankxc)
xc = blankxc
nlxnet = xce.net.eX(n_input = 15, use = [], ueg_limit=True, lob=1.174)
nlcnet = xce.net.eC(n_input = 13, use = [], ueg_limit=True)

nlxc = xce.xc.eXC(grid_models = [nlxnet, nlcnet], level=4)

In [45]:

xc(dms[0], ao_evals[0], gws[0])

spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)


Array(-2.51022075, dtype=float64)

In [24]:
class Band_gap_janak_loss(eqx.Module):
    def __init__(self):
        """
        Initializer for the loss module, which attempts to find loss bang gaps w.r.t. reference

        .. todo: Make more robust for non-local descriptors
        """
        super().__init__()

    def __call__(self, model, ao_eval, gw, dm, eri, mo_occ, hc, s, ogd, refgap, mf, alpha0=0.7):
        """
        Forward pass for loss object

        NOTE: This differs from HoLu loss in that it selects the deepest minimum w.r.t. the LUMO (Fermi energy)

        :param model: The model that will be used in generating the molecular orbital energies ('band' energies)
        :type model: xcquinox.xc.eXC
        :param ao_eval: The atomic orbitals evaluated on the grid for the given molecule
        :type ao_eval: jax.Array
        :param gw: The grid weights associated to the current molecule's grids
        :type gw: jax.Array
        :param dm: Input reference density matrix for use during the one-shot forward pass to generate the new DM
        :type dm: jax.Array
        :param eri: Electron repulsion integrals associated with this molecule
        :type eri: jax.Array
        :param mo_occ: The molecule's molecular orbital occupation numbers
        :type mo_occ: jax.Array
        :param hc: The molecule's core Hamiltonian
        :type hc: jax.Array
        :param s: The molecule's overlap matrix
        :type s: jax.Array
        :param ogd: The original dimensions of this molecule's density matrix, used if padded to constrict the eigendecomposition to a relevant shape
        :type ogd: jax.Array
        :param refgap: The reference gap to optimzie against
        :type refgap: jax.Array
        :param mf: A pyscf(ad) converged calculation kernel if self.level > 3, used for building the CIDER nonlocal descriptors, defaults to None
        :type mf: pyscfad.dft.RKS kernel
        :param alpha0: The mixing parameter for the one-shot density matrix generation, defaults to 0.7
        :type alpha0: float, optional
        :return: Root-squared error between predicted gap (minimum of molecular energies) and the reference
        :rtype: jax.Array
        """
        def janak_theorem_deriv(model, ao_eval, gw, dm, eri, hc, s, ogd, alpha0=0.7):
            def ret_func(mo_occ):
                vgf = lambda x: model(x, ao_eval, gw)
                dmp, moep, mocp = xce.utils.get_dm_moe(dm, eri, vgf, mo_occ, hc, s, ogd, alpha0=alpha0)
                return model(dmp, ao_eval, gw)
        
            return ret_func

        janak_f = janak_theorem_deriv(model, ao_eval, gw, dm, eri, hc, s, ogd, alpha0)
        homo_i = jnp.max(jnp.nonzero(mo_occ, size=dm.shape[0])[0])

        e, derivs = eqx.filter_value_and_grad(janak_f)(mo_occ)

        pred_diff = derivs[homo_i+1] - derivs[homo_i]
        
        loss = jnp.sqrt( (pred_diff - refgap)**2)
        # print(loss)
        return loss


In [25]:
def janak_theorem_deriv(model, ao_eval, gw, dm, eri, moocc, hc, s, ogd, alpha0=0.7):
    def ret_func(mo_occ):
        vgf = lambda x: model(x, ao_eval, gw)
        dmp, moep, mocp = xce.utils.get_dm_moe(dm, eri, vgf, mo_occ, hc, s, ogd, alpha0=alpha0)
        return model(dmp, ao_eval, gw)

    return ret_func
    
    
    

In [26]:
checkd = janak_theorem_deriv(xc, ao_evals[0], gws[0], dms[0], eris[0], mo_occs[0], hcs[0], ss[0], ogds[0], alpha0=0.7)
eqx.filter_value_and_grad(checkd)(mo_occs[0])

spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)


(Array(-3.11735723, dtype=float64),
 Array([-0.76418695, -0.23435005, -0.51950138, -0.51950137,  0.        ,
         0.        ,  0.        ,  0.        ], dtype=float64))

In [48]:
xct = xce.train.xcTrainer(model=xc, optim=optax.adamw(1e-2), steps=100, loss = Band_gap_janak_loss(), do_jit=True)
newm = xct(1, xct.model, ao_evals, gws, dms, eris, mo_occs, hcs, ss, ogds, [1.17], mfs)

Epoch 0
Epoch 0 :: Batch 0/1
spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)



KeyboardInterrupt



In [39]:
e1 = nlxc(dms[0], ao_evals[0], gws[0], mfs[0])
e2 = newm(dms[0], ao_evals[0], gws[0], mfs[0])

spin_scaling = True; input descr to exc shape: (50653, 15)
eX.__call__, rho shape: (50653, 15)
spin_scaling = False; input descr to exc shape: (50653, 13)
eC.__call__, rho shape: (50653, 13)
spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)


In [40]:
e1, e2

(Array(-2.57218399, dtype=float64), Array(-2.90765126, dtype=float64))

In [41]:
vgf1 = lambda x: xc(x, ao_evals[0], gws[0], mfs[0])
vgf2 = lambda x: newm(x, ao_evals[0], gws[0], mfs[0])
dm1, moe1, moc1 = xce.utils.get_dm_moe(dms[0], eris[0], vgf1, mo_occs[0], hcs[0], ss[0], ogds[0])
dm2, moe2, moc2 = xce.utils.get_dm_moe(dms[0], eris[0], vgf2, mo_occs[0], hcs[0], ss[0], ogds[0])

spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()
spin_scaling = True; input descr to exc shape: (2, 50653, 3)
eX.__call__, rho shape: (2, 50653, 3)
spin_scaling = False; input descr to exc shape: (50653, 4)
eC.__call__, rho shape: (50653, 4)
[8] (8,)
(8, 8) (8, 8)
Spin unpolarized make_rdm1()


In [42]:
print(moe1 - moe1[mf.mol.nelectron//2-1])
print(moe2 - moe2[mf.mol.nelectron//2-1])

[-6.57726192e-01 -7.60975504e-02 -3.70073737e-07  0.00000000e+00
  1.02667394e-01  1.02667614e-01  3.18068487e-01  5.38626343e-01]
[-6.86420513e-01 -2.79444128e-02 -3.70097756e-07  0.00000000e+00
  1.49307013e-01  1.49307234e-01  3.48533950e-01  5.46699981e-01]


In [32]:
mf.mo_energy

array([-0.24878937,  0.22904513,  0.22904513,  0.22904513,  0.43234039,
        0.43234039,  0.43234039,  0.45784889])

In [33]:
# vbmax = -99
for en in b1[0]:
    vb_k = en[cell.nelectron//2-1]
    print('This vb_k', vb_k)
    if vb_k > vbmax:
        vbmax = vb_k
e_kn = [en - vbmax for en in b1[0]]

NameError: name 'b1' is not defined

In [ ]:
e_kn

In [ ]:
mf2 = scfp.RHF(cell)
e2 = mf2.kernel()

In [ ]:
cell.nelectron//2-1

In [ ]:
t1 = mf2.mo_energy 
t2 = mf2.mo_energy - mf2.mo_energy[cell.nelectron//2-1]

In [ ]:
t2[jnp.where(abs(t2[jnp.where( (t2 < 0) )[0]]) > 1e-4)[0]]

In [ ]:
dm2 = mf2.make_rdm1()

In [ ]:
dmk = mf.make_rdm1()

In [ ]:
mpr = MPRester(api_key = '')
mpid = 'mp-149'
# ret = mpr.get_bandstructure_by_material_id(mpid)
# rets = ret.structure.as_dict()
ret = mpr.get_structure_by_material_id(mpid, conventional_unit_cell=False)
rets = ret.as_dict()
at_coor_xyz = [ (i['species'][0]['element'], [-j for j in i['xyz']]) for i in rets['sites']]
at_coor_abc = [ (i['species'][0]['element'], [rets['lattice']['a']*j for j in i['abc']]) for i in rets['sites']]
cella = -np.asarray(rets['lattice']['matrix'])

In [ ]:
at_coor_xyz, cella

In [ ]:
cell = gtop.Cell()
cell.atom = at_coor_xyz
cell.a = cella.T
cell.basis = 'gth-szv'
cell.pseudo = 'gth-pade'
cell.exp_to_discard = 0.1
cell.build()
kpts = cell.make_kpts([2,2,2])
mf = scfp.KRHF(cell, kpts=kpts)
e = mf.kernel()

In [ ]:
cella